<a href="https://colab.research.google.com/github/djmayo04/AI_Mini-Project2/blob/main/notebooks/AI_MP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain-groq groq

In [ ]:
import os

REPO_URL = "https://github.com/djmayo04/AI_Mini-Project2.git"
REPO_DIR = "AI_Mini-Project2"

%cd /content

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}
else:
    %cd {REPO_DIR}
    !git pull
    %cd /content

%cd {REPO_DIR}
!ls

In [ ]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
print(os.environ.get("GROQ_API_KEY") is not None)

In [ ]:
!pip install --upgrade pip
!pip install -U langchain langchain-community langchain-text-splitters
!pip install chromadb

In [ ]:
# load the data by looping through each file and split it into chunks with langchain_text_splitters done in llmhelper.py
from langchain_community.document_loaders import TextLoader
from llm_helper import chunk_split

all_docs = []

for file in [
    "data/Assignment1.txt",
    "data/Rubric.txt",
    "data/Syllabus.txt",
    "data/faq.txt"
]:
    loader = TextLoader(file)
    all_docs.extend(loader.load())

chunks = chunk_split(all_docs)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Embedding model
embeddings = HuggingFaceEmbeddings()

# Transforms chunks into vectors for storage using HuggingFaceEmbeddings model
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)


In [ ]:
from tools import set_vectorstore

# Set/export the vectorstore to tools.py

set_vectorstore(vectorstore)

In [ ]:
# Test of similarity_search in the vectorstore
docs = vectorstore.similarity_search("assignment deadline", k=2)
print(docs[0].page_content)

In [ ]:
from llm_helper import get_completion
from agent import run_agent

print(get_completion("Say hello in one sentence."))
print(run_agent("What does the syllabus say about deadlines?"))